# CS3315 Final Project
# Authors: Cameron Woods and Micky Hall 

## Introduction 

Throughout this notebook we will be attempting to create a model using supervised learning techniques that can accurately predict the price of an airbnb for a night. We will be using a dataset from kaggle that includes 226,029 rows. Each row represents an individual airbnb listing and includes 15 features and a label which will be discussed in depth at a later point in this notebook. 

### General Process 

We began this project by taking all of our unprocessed data and attempting to fit it into a bare bones model using linear regression to see how well it would perform. We then analysed the results and looked for reasons in the large skew in our data. We then went back and looked at the data as a whole and began to munge our data into a more palatable set for future models. We then checked for any increase in performance from our model. We then began to look into feature engineering and hyper parameter tuning for a greater fit. We slowly worked our way to a better model. We then decided to attempt to run our data in a neural network and see how well that would predict our label. All in all our models ended up predicting with a mse of %%%.  

In [37]:
import pandas as pd 

df = pd.read_csv("AB_US_2020.csv")


In [38]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226029 entries, 0 to 226028
Data columns (total 17 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              226029 non-null  int64  
 1   name                            226001 non-null  object 
 2   host_id                         226029 non-null  int64  
 3   host_name                       225996 non-null  object 
 4   neighbourhood_group             110185 non-null  object 
 5   neighbourhood                   226029 non-null  object 
 6   latitude                        226029 non-null  float64
 7   longitude                       226029 non-null  float64
 8   room_type                       226029 non-null  object 
 9   price                           226029 non-null  int64  
 10  minimum_nights                  226029 non-null  int64  
 11  number_of_reviews               226029 non-null  int64  
 12  last_review     

In [39]:
del df["name"]
del df["host_name"]
del df["neighbourhood_group"]
del df["city"]
del df["neighbourhood"]
del df["last_review"]


df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226029 entries, 0 to 226028
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              226029 non-null  int64  
 1   host_id                         226029 non-null  int64  
 2   latitude                        226029 non-null  float64
 3   longitude                       226029 non-null  float64
 4   room_type                       226029 non-null  object 
 5   price                           226029 non-null  int64  
 6   minimum_nights                  226029 non-null  int64  
 7   number_of_reviews               226029 non-null  int64  
 8   reviews_per_month               177427 non-null  float64
 9   calculated_host_listings_count  226029 non-null  int64  
 10  availability_365                226029 non-null  int64  
dtypes: float64(3), int64(7), object(1)
memory usage: 19.0+ MB


In [40]:
del df["reviews_per_month"]

In [41]:
def oneHot(category, hot):
    if category == hot:
        return 1
    else:
        return 0

dict={}
for room in df['room_type'].tolist():
    dict[room]=1
    
for room in dict.keys():
    df[room] = df['room_type'].apply(oneHot, hot=room)

df = df.drop(['room_type'],axis=1)
df.head()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,Private room,Entire home/apt,Hotel room,Shared room
0,38585,165529,35.65146,-82.62792,60,1,138,1,0,1,0,0,0
1,80905,427027,35.59779,-82.55540,470,1,114,11,288,0,1,0,0
2,108061,320564,35.60670,-82.55563,75,30,89,2,298,0,1,0,0
3,155305,746673,35.57864,-82.59578,90,1,267,5,0,0,1,0,0
4,160594,769252,35.61442,-82.54127,125,30,58,1,0,1,0,0,0


In [42]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer


#imp = SimpleImputer(missing_values=np.nan, strategy='mean')
#imp.fit(df)
#transformed_df = imp.transform(df)
#new_df = pd.DataFrame(transformed_df,columns=df.columns,index=df.index)

X = df.drop(["price"],axis=1)
y = df["price"]

#imputer = KNNImputer(n_neighbors=2, weights="uniform")
#imputer.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)


sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)

In [110]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
sgd_reg = LinearRegression()
sgd_reg.fit(X_train, y_train) # continues where it left off



LinearRegression()

In [111]:
from math import sqrt
y_val_predict = sgd_reg.predict(X_val)
val_error = sqrt(mean_squared_error(y_val, y_val_predict))
print(val_error)

171005.66879026606


In [126]:
y.mean()
y.std()
y.describe()
y_val_predict.sort()
y_val_predict[658]

0.0799059892283367